# Machine Learning Semester Project
## Murtaza Hussain (29449) and Muhammad Asad ur Rehman (29456)

### Class Imbalance Problem

The below code solves the prevalent problem of imbalanced dataset, where one class dominates the dataset as compared to the other. Such is the case for the following dataset for Credit Card Transactions to detect Fraudulent Transactions. We will evaluate the following methods to resolve Class Imbalance:
1. Random Under Sampling
2. Algorithmic Methods (Using Random Forest as well as modifying Class Weights)
3. Anomaly Detection Method

For the following Dataset, we will use the following 5 Algorithms to draw a comparision between different methods:
1. Logistic Regression
2. K-Nearest Neighbors (KNN)
3. Random Forest
4. Support Vector Machines (SVM)
5. Naive Bayes (Gaussian)

In [ ]:
# Import necessary libraries
from sklearn.metrics import make_scorer, recall_score, precision_score
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, OneClassSVM
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.4f}'.format

In [ ]:
# Data Loader loads data from CSV Files
def load_dataset():
    dataset = pd.read_csv("./Source.CreditCardFraud.csv")
    return dataset

# df = load_dataset()

In [ ]:
# This function performs a missing value analysis on each column of the dataset, helps you decide on what to do in cleaning process
def null_check(df):
    null_columns = []
    for column in df.columns:
        print("Column Name:", column)
        print("Column DataType:", df[column].dtype)
        if df[column].dtype != 'float64' and df[column].dtype != 'int64':
            print("Column unique values:", df[column].unique())
        print("Column has null:", df[column].isnull().any())

        
        if df[column].isnull().any() == True:
            print("Column Null Count:", df[column].isnull().sum())
            null_columns.append(column)
        print("\n")
    return null_columns

# null_check(df)

In [ ]:
# This function drops any null columns and missing values
# This is where you decide whether to remove NULL rows (which will reduce the size of Dataset) or remove NULL columns entirely. You can also choose a combination of both.
def clean_data(df, drop_columns, missing_value = False):
    # Remove unnecessary columns
    df.drop(drop_columns, axis=1, inplace=True)
    # Drop rows with any missing values
    if missing_value == False:
        df.dropna(inplace=True)
    else:
        df.fillna(missing_value, inplace=True)
    return df

In [ ]:
# Prints a summary of class instances and distribution
def data_summary(df, target=None):
    if isinstance(df, pd.DataFrame) and target!=None:
        a = df[target].value_counts()
    else:
        a = df.value_counts()
    class0 = format(100 * a[0]/sum(a), ".2f")
    class1 = format(100 * a[1]/sum(a), ".2f")

    meta = pd.DataFrame([{ "%": class0, "count": a[0]},
                         { "%": class1, "count": a[1]}])
    print("\nClass Distribution:\n", meta, "\n")

# data_summary(df,'Class')

In [ ]:
# Transforms categorical and numberical data into numerical data
def transform_data(df):
    # Encode categorical variables
    label_encoder = LabelEncoder()
    print("Categorical columns:", df.select_dtypes(include=['object', 'int64']).columns)
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = label_encoder.fit_transform(df[col])
    
    # Standardize numerical features
    scaler = StandardScaler()
    print("Numerical columns:", df.select_dtypes(include=['float64']).columns)
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    if len(numerical_cols) > 0:
        df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    return df

# df['Class'] = df['Class'].astype(str)
# df = transform_data(df)

In [ ]:
# Runs Baseline Model for All 5 Algorithms
def BaselineRunAll(df, target_name, k=10):

    # Separate features and targets
    X = df.drop(target_name, axis=1)
    y = df[target_name]
    results = []

    print("Class Distribution for Baseline Run:")
    data_summary(y)

    # Initialize the classifiers
    lr_classifier = LogisticRegression(max_iter=1000)
    rf_classifier = RandomForestClassifier()
    knn_classifier = KNeighborsClassifier()
    svm_classifier = SVC(probability=True)
    nb_classifier = GaussianNB()
    

    # Initialize k-fold cross-validation where folds = 10
    k_fold = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)     # The reasoning behind k = 10 is so as to strike a balance between test and train samples of minority class

    # Define a recall and precision scorer specifically focusing on the minority class
    recall_precision_scorer = {'recall': make_scorer(recall_score, pos_label=1), # As the majority class has 99.81% presence, accuracy cannot be used as a metric to evaluate performance
                               'precision': make_scorer(precision_score, pos_label=1)}
    
    classifiers = {
        'Logistic Regression': lr_classifier,
        'Random Forest': rf_classifier,
        'K-Nearest Neighbours': knn_classifier,
        'Support Vector Machines': svm_classifier,
        'Naive Bayes': nb_classifier
    }

    for clf_name, clf in classifiers.items():
        scores = cross_validate(clf, X, y, cv=k_fold, scoring=recall_precision_scorer)
        print(f"{clf_name} Model Training Completed")
        mean_recall = scores['test_recall'].mean()
        mean_precision = scores['test_precision'].mean()

        results.append({
            'Method': 'Baseline',
            'Classifier': clf_name,
            'Class 1 Recall': mean_recall,
            'Class 1 Precision': mean_precision
        })

    df = pd.DataFrame(results)
    return df

# results = BaselineRunAll(df, 'Class')
# print(results)

In [ ]:
# Performs One Class Learning (Majority Class) on all 5 Algorithms
def OneClassLearning(df, target_name, k=10):

    # Separate features and targets
    X = df.drop(target_name, axis=1)
    y = df[target_name]
    results = []

    print("Class Distribution for One Class Learning Majority Class:")
    data_summary(y)

    # Initialize the classifiers
    lr_classifier = LogisticRegression(max_iter=1000)
    rf_classifier = RandomForestClassifier()
    knn_classifier = KNeighborsClassifier()
    svm_classifier = SVC(probability=True)
    nb_classifier = GaussianNB()

    # Separate minority and majority class data
    X_minority = X[y == 1]
    X_majority = X[y == 0]
    y_minority = y[y == 1]
    y_majority = y[y == 0]

    # Initialize k-fold cross-validation where folds = 10 (Will have to change the cross validation technique)
    k_fold = KFold(n_splits=k, shuffle=True, random_state=42)
    majority_indices = k_fold.split(X_majority)

    # Using last fold as the test fold which includes all minority data
    train_indices = [train_index for train_index, _ in majority_indices]
    test_indices = train_indices.pop()  # We use the last fold's train_index as our test set index

    X_train = X_majority.iloc[np.concatenate(train_indices)]
    X_test = pd.concat([X_majority.iloc[test_indices], X_minority])
    y_train = y_majority.iloc[np.concatenate(train_indices)]
    y_test = pd.concat([y_majority.iloc[test_indices], y_minority])

    classifiers = {
        'Logistic Regression': lr_classifier,
        'Random Forest': rf_classifier,
        'K-Nearest Neighbours': knn_classifier,
        'Support Vector Machines': svm_classifier,
        'Naive Bayes': nb_classifier
    }

    # Train on majority class only
    majority_class = (y == 0)
    X_majority = X[majority_class]
    y_majority = y[majority_class]

    # Manual Training and evaluating classifiers
    for clf_name, clf in classifiers.items():
        clf.fit(X_train, y_train)
        print(f"{clf_name} Model Training Completed")
        y_pred = clf.predict(X_test)
        recall = recall_score(y_test, y_pred, pos_label=1)
        precision = precision_score(y_test, y_pred, pos_label=1)

        results.append({
            'Method': 'Undersampling + SMOTE',
            'Classifier': clf_name,
            'Class 1 Recall': recall,
            'Class 1 Precision': precision
        })
    
    df = pd.DataFrame(results)
    return df

# results = RandomSamplingSMOTE(df, 'Class')
# print(results)

In [ ]:
# Performs One Class Learning (Majority Class) using special methods
def OneClassLearningSpecialMethods(df, target_name, k=10):

    # Separate features and targets
    X = df.drop(target_name, axis=1)
    y = df[target_name]
    results = []

    print("Class Distribution for One Class Learning Special Methods:")
    data_summary(y)

    # Initialize the classifiers
#    lr_classifier = LogisticRegression(max_iter=1000) # Has no such implementation
#    rf_classifier = RandomForestClassifier() # Has no such implementation
    knn_classifier = LocalOutlierFactor(novelty=True) # A special implementation of KNN for One Class Learning
    svm_classifier = OneClassSVM(kernel='rbf', gamma='auto') # A special implementation of SVM for One Class Learning
#    nb_classifier = GaussianNB() # Has no such implementation

    # Initialize k-fold cross-validation where folds = 10
    k_fold = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    # Define a recall and precision scorer specifically focusing on the minority class
    recall_precision_scorer = {'recall': make_scorer(recall_score, pos_label=1), 
                               'precision': make_scorer(precision_score, pos_label=1)}

    classifiers = {
        # 'Logistic Regression': lr_classifier,
        # 'Random Forest': rf_classifier,
        'K-Nearest Neighbours': knn_classifier,
        'Support Vector Machines': svm_classifier,
        # 'Naive Bayes': nb_classifier
    }

    # Train on majority class only
    majority_class = (y == 0)
    X_majority = X[majority_class]
    y_majority = y[majority_class]

    for clf_name, clf in classifiers.items():
        scores = cross_validate(clf, X_majority, y_majority, cv=k_fold, scoring=recall_precision_scorer, return_train_score=False)
        print(f"{clf_name} Model Training Completed")
        mean_recall = scores['test_recall'].mean()
        mean_precision = scores['test_precision'].mean()

        results.append({
            'Method': 'Undersampling + SMOTE',
            'Classifier': clf_name,
            'Class 1 Recall': mean_recall,
            'Class 1 Precision': mean_precision
        })
    
    df = pd.DataFrame(results)
    return df

# results = RandomSamplingSMOTE(df, 'Class')
# print(results)

In [ ]:
# Plot a Model vs Recall graph for Classification Dataset for Each Method
def plot_model_recall_graph(df):
    plt.figure(figsize=(10, 6))
    
    # Plotting lines for each Method
    sns.lineplot(data=df, x='Classifier', y='Class 1 Recall', hue='Method', marker='o')

    plt.title('Classifier vs Recall')
    plt.xlabel('Classifier')
    plt.ylabel('Recall')
    plt.legend(title='Method')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot a Model vs Precision graph for Classification Dataset for Each Method
def plot_model_precision_graph(df):
    plt.figure(figsize=(10, 6))
    
    # Plotting lines for each Method
    sns.lineplot(data=df, x='Classifier', y='Class 1 Precision', hue='Method', marker='o')

    plt.title('Classifier vs Precision')
    plt.xlabel('Classifier')
    plt.ylabel('Precision')
    plt.legend(title='Method')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
def master_workflow():
    # Load Dataset
    df = load_dataset()
    # No need for Data Cleaning and EDA as Data is already clean
    # Evaluate Class Distribution of the cleaned Dataset
    data_summary(df,'Class')
    # Transform and Encode Data
    df['Class'] = df['Class'].astype(str)
    df = transform_data(df)
    # Run Baseline Models using k = 10
    baseline_results = BaselineRunAll(df, 'Class')
    # Evaluate Models using SMOTE Oversampling Technique
    smote_results = RandomSamplingSMOTE(df, 'Class')
    # Evaluate Models using ADASYN Oversampling Technique
    adasyn_results = RandomSamplingADASYN(df, 'Class')
    # Concatenate the results
    results_df = pd.concat([baseline_results, smote_results, adasyn_results])
    # Print results
    print(results_df)
    results_df.to_csv('SamplingMethodsResults.csv', index=False)
    # Plot a Classifier vs Recall Graph -> To evaluate how well the model is performing to detect the fraudulent transactions (minority class)
    plot_model_recall_graph(results_df)
    # Plot a Classifier vs Precision Graph -> To evaluate how precise the model is to detect the minority class (can be used as a secondary metric for evaluation)
    plot_model_precision_graph(results_df)

In [ ]:
# As the pipeline was not running in one go, we had to split it into smaller parts
# master_workflow() 

### Broken down pipeline:

In [ ]:
# Load Dataset
df = load_dataset()
# No need for Data Cleaning and EDA as Data is already clean
# Evaluate Class Distribution of the cleaned Dataset
data_summary(df,'Class')
# Transform and Encode Data
df['Class'] = df['Class'].astype(str)
df = transform_data(df)

In [ ]:
# Run Baseline Models using k = 10
baseline_results = BaselineRunAll(df, 'Class')

In [ ]:
# Evaluate Models using SMOTE Oversampling Technique
one_class_results = OneClassLearning(df, 'Class')

In [ ]:
# Evaluate Models using ADASYN Oversampling Technique
adasyn_results = RandomSamplingADASYN(df, 'Class')

In [ ]:
# Concatenate the results
results_df = pd.concat([baseline_results, smote_results, adasyn_results])
# Print results
print(results_df)
results_df.to_csv('SamplingMethodsResults.csv', index=False)
# Plot a Classifier vs Recall Graph -> To evaluate how well the model is performing to detect the fraudulent transactions (minority class)
plot_model_recall_graph(results_df)
# Plot a Classifier vs Precision Graph -> To evaluate how precise the model is to detect the minority class (can be used as a secondary metric for evaluation)
plot_model_precision_graph(results_df)